In [1]:
class DiscriminatorACGAN(nn.Module):
    def __init__(self, nz=1, nc=1, input_size=32, class_num=10):
        super(DiscriminatorACGAN, self).__init__()
        self.nz = nz
        self.nc = nc
        self.input_size = input_size
        self.class_num = class_num

        self.conv = nn.Sequential(
            nn.Conv2d(self.nz, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(128 * (self.input_size // 4) * (self.input_size // 4), 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
        )
        self.dc = nn.Sequential(
            nn.Linear(1024, self.nc),
            nn.Sigmoid(),
        )
        self.cl = nn.Sequential(
            nn.Linear(1024, self.class_num),
        )
        initialize_weights(self)

    def forward(self, input):
        x = self.conv(input)
        x = x.view(-1, 128 * (self.input_size // 4) * (self.input_size // 4))
        x = self.fc1(x)
        d = self.dc(x)
        c = self.cl(x)

        return d, c

NameError: name 'nn' is not defined

In [ ]:
# networks init ACGAN

input_size = 28
z_dim = 62
class_num = 10
sample_num = class_num ** 2
batch_size = 64

# networks init
lrG= 0.0002
lrD=0.0002
beta1=0.5
beta2=0.99

G = Generator(nz=z_dim, nc=1, input_size=input_size, class_num=class_num)
D = DiscriminatorACGAN(nz=1, nc=1, input_size=input_size, class_num=class_num)

print(G)
print(D)

optimizerG = optim.Adam(G.parameters(), lr=lrG, betas=(beta1, beta2))
optimizerD = optim.Adam(D.parameters(), lr=lrD, betas=(beta1, beta2))

# Define loss

BCE_loss = nn.BCELoss()
CE_loss = nn.CrossEntropyLoss()

In [ ]:
def train_acgan(D,G,d_loss_total,g_loss_total,y_real_,y_fake_,y_vec_,x_,z_,y_fill_):
  
        ############################
        # (1) Update D network: )
        ###########################
        # Train with all-real batch
    
        optimizerD.zero_grad()

        # Real images
        D_real, C_real = D(x_)
        D_real_loss = BCE_loss(D_real, y_real_)
        C_real_loss = CE_loss(C_real, torch.max(y_vec_, 1)[1])

        # Fake images
        G_ = G(z_, y_vec_)
        D_fake, C_fake = D(G_)
        D_fake_loss = BCE_loss(D_fake, y_fake_)
        C_fake_loss = CE_loss(C_fake, torch.max(y_vec_, 1)[1])

        # Discriminator total loss

        D_loss = D_real_loss + C_real_loss + D_fake_loss + C_fake_loss
        D_loss.backward()
        optimizerD.step()

        d_loss_total += D_loss.item()


        # Update Generator network
        optimizerG.zero_grad()

        # Fake images a
        G_ = G(z_, y_vec_)
        D_fake, C_fake = D(G_)

        # Generator Loss 
        G_loss = BCE_loss(D_fake, y_real_)
        C_fake_loss = CE_loss(C_fake, torch.max(y_vec_, 1)[1])

        # Total loss for Generator
        G_loss += C_fake_loss
        G_loss.backward()
        optimizerG.step()

        g_loss_total += G_loss.item()


        # Calculate real images for KID 
        real_images = torch.cat([x_, y_fill_], 1)  # Concatenate x_ and y_fill_
        fake = G_

        return D_loss, G_loss, real_images, fake

In [ ]:
d_losses_infogan_acgan,g_losses_infogan_acgan , SSIM_SCORES_infogan_acgan, KID_SCORES_infogan_acgan = train_model("InfoGAN-ACGAN",G,D,"10")

In [ ]:
plot_losses(d_losses_infogan_acgan, g_losses_infogan_acgan,"InfoGAN-ACGAN")
ssim_plot(SSIM_SCORES_infogan_acgan,"InfoGAN-ACGAN")
kid_plot(KID_SCORES_infogan_acgan,"InfoGAN-ACGAN")